In [ ]:
import pandas as pd
import numpy as np
from google.colab import files

def validate_inputs(data, w, imp):
    if data.shape[1] < 3:
        raise ValueError("Dataset must contain at least 3 columns")

    try:
        numeric = data.iloc[:,1:].apply(pd.to_numeric)
    except:
        raise ValueError("All criteria columns must be numeric")

    w_list = w.split(",")
    i_list = imp.split(",")

    if len(w_list) != numeric.shape[1]:
        raise ValueError("Weights count mismatch")

    if len(i_list) != numeric.shape[1]:
        raise ValueError("Impacts count mismatch")

    for val in i_list:
        if val not in ["+","-"]:
            raise ValueError("Impacts must be + or -")

    return numeric, np.array(w_list,dtype=float), i_list


def normalize_matrix(mat):
    denom = np.sqrt((mat**2).sum(axis=0))
    return mat/denom


def ideal_values(weighted, impacts):
    best=[]
    worst=[]
    for col,impact in enumerate(impacts):
        column=weighted.iloc[:,col]
        if impact=="+":
            best.append(column.max())
            worst.append(column.min())
        else:
            best.append(column.min())
            worst.append(column.max())
    return np.array(best),np.array(worst)


def distance_calc(weighted,best,worst):
    d_pos=np.sqrt(((weighted-best)**2).sum(axis=1))
    d_neg=np.sqrt(((weighted-worst)**2).sum(axis=1))
    return d_pos,d_neg


def compute_topsis(df,weights,impacts):

    matrix,weights,impacts=validate_inputs(df,weights,impacts)

    norm=normalize_matrix(matrix)
    weighted=norm*weights

    best,worst=ideal_values(weighted,impacts)

    d_pos,d_neg=distance_calc(weighted,best,worst)

    scores=d_neg/(d_pos+d_neg)

    df["Score"]=scores
    df["Rank"]=df["Score"].rank(method="max",ascending=False).astype(int)

    return df.sort_values("Rank")


print("Upload CSV file")
uploaded=files.upload()

filename=list(uploaded.keys())[0]
data=pd.read_csv(filename)

weights=input("Weights: ")
impacts=input("Impacts: ")

result=compute_topsis(data,weights,impacts)

result.to_csv("topsis_output.csv",index=False)

print("\nFinal Ranking:")
display(result)

files.download("topsis_output.csv")